In [1]:
import sqlite3
import glob
import urllib.parse
import records
import requests
from bs4 import BeautifulSoup
import MeCab as mc
import re
from collections import Counter
from wordcloud import WordCloud
from matplotlib import pyplot as plt
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



In [2]:
conn = None

# データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect("./sample.db")
    
# コネクションを断つ
def close():
    conn.close()
    
# テーブル作成
def create_table():
    # DROPは消す.
    conn.execute("DROP TABLE IF EXISTS countries")
    conn.execute("""CREATE TABLE countries (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            country_name TEXT,
            content TEXT
    )""")
    
# loadは積むという意味
def load(name, content):
    conn.execute(
        "INSERT INTO Countries (country_name, content) VALUES (?,?)", (name, content))
    conn.commit()
    
def get(doc_id, fl):
    row_ls = conn.execute(
        "SELECT {} FROM docs WHERE id = ?".format(",".join(fl)),(doc_id,)).fetchone()
    row_dict = {}
    for key, value in zip(fl, row_ls):
        row_dict[key] = value
    return row_dict

def get_all_ids(limit, offset=0):
    return [record[0] for record in conn.execute("SELECT id FROM docs LIMIT ? OFFSET ?",(limit, offset))]

In [10]:
url = "https://ja.wikipedia.org"
r = requests.get(url+"/wiki/国の一覧")

html_soup = BeautifulSoup(r.text, "html.parser")
table = html_soup.find("table",class_= "wikitable")
connect()
create_table()

for tr in table.find_all("tr"):
    link = tr.find_all("a")
    if link:
        country_name = link[1].text
        country_link = link[1].get("href")
        
        res = requests.get(url+country_link)
        load(country_name,res.text)
close()

In [9]:
def preprocess():
    con = None
    con = sqlite3.connect("./sample.db")
    cursor = con.cursor()
    map_in=input()
    res = cursor.execute("SELECT content FROM Countries WHERE country_name = ?",[map_in])
    # print(res)
    # bs = BeautifulSoup(cursor.fetchall()[0][0],"html.parser")
    bs = BeautifulSoup(cursor.fetchone()[0],"html.parser")
    
    
    div_all = bs.find("div", id = "content").text
    # print(re.sub("[\n,\xa0^,\<http\S*]","",div_all))
    del_n = re.sub("[\n,\xa0^]","",div_all)
    del_wiki = re.sub("出典|編集|参照|ページ|cs|閲覧|wiki[pm]edia|parser|CentralStatistiOffice|font|url|commons|alt|svg","",del_n)
    del_wiki2 = re.sub("left|background|kern|km|right|emcenter|position|upload|output|thumb|mw|png|inherit|org|px|citation","",del_wiki)
    del_wiki3 = re.sub("emcenter|repeat|それ|一覧|略称|以上|こと|表記|もの|記事|以下|ため|直接|出版|構成|ほか|これ|よう|識別子|指定","",del_wiki2)
    del_all = re.sub("https?\S*\s","",del_wiki3)
    del_space = re.sub(" ","",del_all)
    # print(del_space)
    return del_space

In [10]:
def mecab_analysis(texts):
    t = mc.Tagger("")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞', '形容詞']:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [11]:
data = []
doc_ids = []
text1 = " ".join(mecab_analysis(preprocess()))
text2 = " ".join(mecab_analysis(preprocess()))
data=[text1,text2]
print(data)

アイスランド共和国
アイルランド
['アイスランド : フリー 百科 事典 ウィキペディア Wikipedia ナビゲーション 移動 検索 移動 アイスランド 共和 国 Ísland 国旗 ）( 国 章 ) 国 標語 国歌 Lofsöngur アイスランド 語 賛美 歌 公用 語 アイスランド 語 首都 レイキャビク 最大 都市 レイキャビク 政府 大統領 グズニ・ヨハンネソン 首相 カトリーン・ヤコブスドッティル 面積 総計 1030002 105 位 水面 積 率 2 . 7 % 人口 総計 2018 年 355620 人 170 位 人口 密度 3 人 / 2 GDP 自国 通貨 表示 合計 2008 年 14650 億 [ 1 ] アイスランド ・ クローナ GDP ( MER ) 合計 2008 年 xxxxxx ドル 91 位 GDP ( PPP ) 合計 xxxx 年 xxxxxx ドル （??? 位 1 人 あたり 40024 [ 1 ] ドル 独立 デンマーク 1944 年 6 月 17 日 通貨 アイスランド ・ クローナ ( ISK ) 時間 帯 UTC ± 0 DST なし ISO 3166 - 1 IS / ISLccTLD . is 国際 電話 番号 354 アイスランド 北 ヨーロッパ 北大西洋 上 位置 共和 制 国家 首都 レイキャビク 人口 337610 人 目次 1 概要 2 国名 3 歴史 3 . 1 植民 自然 影響 3 . 2 新天地 自治 終焉 3 . 3 独立 運動 4 政治 4 . 1 EU 加盟 問題 4 . 1 . 1 背景 4 . 1 . 2 加盟 申請 4 . 1 . 3 加盟 交渉 開始 4 . 1 . 4 加盟 申請 取り下げ 4 . 2 エネルギー 政策 4 . 3 捕鯨 5 軍事 6 地方 行政 区分 6 . 114 市 7 地理 7 . 1 火山 7 . 1 . 1 主 湖 7 . 1 . 2 主 山 7 . 2 気候 7 . 3 都市 交通 8 経済 8 . 1 国家 経済 8 . 1 . 1 金融 危機 以前 8 . 1 . 2 金融 危機 以後 8 . 2 資源 8 . 3 漁業 8 . 4 エネルギー 8 . 5 製造 業 8 . 6 商業 8 . 7 貿易 9 国民 9 . 1 人口 統計

In [12]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [27]:
ice = preprocess() 
ice_mecab = mecab_analysis(ice)
# ire = mecab_analysis(preprocess())
dictionary = Dictionary([ice_mecab])
# dictionary.filter_extremes(no_below=2,no_above=0.3)
corpus = [dictionary.doc2bow(doc) for doc in [ice_mecab]]

ida = LdaModel(corpus, num_topics=5, id2word=dictionary, passes=2)

アイスランド共和国


In [29]:
for topic in ida.show_topics(num_topics=-1,num_words=10):
    print("topic id:{0[0]}, word={0[1]}".format(topic))

topic id:0, word=0.015*"アイスランド" + 0.009*"年" + 0.008*"." + 0.006*"月" + 0.006*"[]" + 0.005*"1" + 0.004*"加盟" + 0.004*"日" + 0.004*"2" + 0.004*"3"
topic id:1, word=0.010*"アイスランド" + 0.006*"年" + 0.004*"月" + 0.004*"日" + 0.004*"[]" + 0.004*"1" + 0.003*"加盟" + 0.003*"(" + 0.003*"[" + 0.003*"."
topic id:2, word=0.017*"アイスランド" + 0.010*"年" + 0.007*"月" + 0.006*"." + 0.005*"日" + 0.005*"1" + 0.005*"[]" + 0.004*"3" + 0.004*"国" + 0.004*"加盟"
topic id:3, word=0.030*"アイスランド" + 0.019*"年" + 0.014*"." + 0.012*"月" + 0.010*"1" + 0.010*"日" + 0.009*"[]" + 0.009*"加盟" + 0.008*"(" + 0.007*"3"
topic id:4, word=0.015*"アイスランド" + 0.012*"年" + 0.008*"月" + 0.007*"." + 0.005*"日" + 0.005*"[]" + 0.005*"1" + 0.004*"3" + 0.004*"[" + 0.004*"("


In [34]:
vectorizer = TfidfVectorizer(analyzer="word", max_df = 0.9)
vecs = vectorizer.fit_transform(data)
vectorizer.get_feature_names()
# for doc_id, vec in zip(doc_ids, vecs.toarray()):
#     meta_info = json.loads(datastore.get(doc_id, ["meta_info"])["meta_info"])
#     title = meta_info["title"]
#     print(doc_id, title)
    
#     for w_id, tfidf in sorted(enumerate(vec), key=lambda x: x[1],reverse=True)[:10]:
#         lemma = vectorizer.get_feature_names()[w_id]
#         print("\t{0:s}: {1:f}".format(lemma, tfidf))

    

['aa',
 'abcde',
 'abcdimfdataandstatisti',
 'abhehataglanceoecd',
 'abimfdataandstatisti',
 'afpbbnews',
 'amhránnabhfiann',
 'annamjöll',
 'anuaimh',
 'axt',
 'aylegaltownanditsenvirons',
 'aysintherepublicofireland',
 'banggang',
 'bankholiday',
 'bennihemmhemm',
 'björk',
 'bloomberg',
 'bnf',
 'border',
 'bottomdotted',
 'bp',
 'brianboru',
 'cabcdefnationalgeographic',
 'category',
 'ccnnnarandlnkcgesudocviaf',
 'ce',
 'census',
 'centralstatistioffice',
 'ceo',
 'cfc',
 'chapt',
 'christmas',
 'cite',
 'code',
 'color',
 'com',
 'cork',
 'county',
 'cursorhelp',
 'dagurkári',
 'dalegaltownanditsenvirons',
 'displaynone',
 'dna',
 'doi',
 'dáiléireann',
 'dóttir',
 'eastermonday',
 'ec',
 'economist',
 'eea',
 'eiðursmáriguðjohnsen',
 'em',
 'emcenter',
 'ementdublincityandsuburbs',
 'ementgalwaycityandsuburbs',
 'en',
 'enceofaliberalpaulkrugman',
 'entlimerickcityandsuburbs',
 'error',
 'et',
 'eur',
 'fanicelanderandaforeigner',
 'fesi',
 'fifa',
 'format',
 'freea',
 'friðrik

In [ ]:
sim=cosine_similarity(vecs)
docs=zip(doc_ids,sim[0])
for doc_id, 